# Distribution Scores
This notebook looks at the distribution of the scores depending on the label. This is done using a plot, the divergence measure, and the lift score. 

In [ ]:
# LOAD MODULES
# Standard library
import os
import sys

# Third party
from tqdm import tqdm

# NOTE: Your script is not in the root directory. We must hence change the system path
DIR = "../"
os.chdir(DIR)
sys.path.append(DIR)

In [ ]:
from src.data.pattern_construction import define_ML_labels, summarise_ML_labels
from src.methods.gargaml_scores import define_gargaml_scores
import pandas as pd

In [ ]:
dataset = "HI-Small"  
directed = False
score_type = "basic" # basic or weighted_average

str_directed = "directed" if directed else "undirected"
results_df_measures = pd.read_csv("results/"+dataset+"_GARGAML_"+str_directed+".csv")

results_df = define_gargaml_scores(results_df_measures, directed=directed, score_type=score_type)

transactions_df_extended, pattern_columns = define_ML_labels(
    path_trans = "data/"+dataset+"_Trans.csv",
    path_patterns = "data/"+dataset+"_Patterns.txt"
)

laundering_combined, _, _ = summarise_ML_labels(transactions_df_extended,pattern_columns)

In [ ]:
from_data = transactions_df_extended[["Account", "From Bank"]].drop_duplicates()
from_data.columns = ["Account", "Bank"]
to_data = transactions_df_extended[["Account.1", "To Bank"]].drop_duplicates()
to_data.columns = ["Account", "Bank"]
total_data = pd.concat([from_data, to_data], axis=0).drop_duplicates()
total_data.shape

## Single Analysis
This part is used to illustrate the code on one example, using a single column and a cut-off for the labels. 

In [ ]:
column = "SCATTER-GATHER"
laundering_combined[[column]].hist(log=True)

In [ ]:
cutoff = 0.2
laundering_combined["Label"] = ((laundering_combined[column]>cutoff)*1).values

labels = []
for node in results_df.index:
    label = int(laundering_combined.loc[node]["Label"])
    labels.append(label)

results_df["Label"] = labels

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Filter the DataFrame by label
label_0 = results_df[results_df["Label"] == 0]["GARGAML"]
label_1 = results_df[results_df["Label"] == 1]["GARGAML"]

# Calculate the bin edges
all_data = np.concatenate([label_0, label_1])
bins = np.histogram_bin_edges(all_data, bins=20)

# Plot histograms
plt.figure(figsize=(10, 5))

# Plot histogram for label 0
plt.hist(label_0, bins=bins, alpha=0.5, label='Label 0', density=True)

# Plot histogram for label 1
plt.hist(label_1, bins=bins, alpha=0.5, label='Label 1', density=True)

# Add labels and title
plt.xlabel('GARGAML')
plt.ylabel('Relative Frequency')
plt.title('Distribution of GARGAML Scores by Label: "'+ column +'" at cut-off '+ str(cutoff))
plt.legend()

# Show plot
plt.savefig("results/"+dataset+"_GARGAML_"+str_directed+"_histogram_"+str(int(cutoff*100))+".pdf")

In [ ]:
# Divergence measure
mean_0 = np.mean(label_0)
variance_0 = np.var(label_0)
mean_1 = np.mean(label_1)
variance_1 = np.var(label_1)

divergence = (mean_0 - mean_1)**2 + 0.5*(variance_0 + variance_1)

print("Divergence measure: ", divergence)

In [ ]:
def lift_curve_values(y_val, y_pred, steps):
    vals_lift = [] #The lift values to be plotted

    df_lift = pd.DataFrame()
    df_lift['Real'] = y_val
    df_lift['Pred'] = y_pred
    df_lift.sort_values('Pred',
                        ascending=False,
                        inplace=True)

    global_ratio = df_lift['Real'].sum() / len(df_lift['Real'])

    for step in steps:
        data_len = int(np.ceil(step*len(df_lift)))
        data_lift = df_lift.iloc[:data_len, :]
        val_lift = data_lift['Real'].sum()/data_len
        vals_lift.append(val_lift/global_ratio)

    return(vals_lift)

In [ ]:
values = np.linspace(0.01, 1, 100)
lift = lift_curve_values(results_df["Label"], results_df["GARGAML"], values)
plt.plot(values, lift)
plt.title('Lift curve of GARGAML Scores by Label: "'+ column +'" at cut-off '+ str(cutoff))
plt.xlabel('Proportion of data')
plt.ylabel('Lift')
plt.savefig("results/"+dataset+"_GARGAML_"+str_directed+"_lift_curve_"+str(int(cutoff*100))+".pdf")

## Multiple Results
This part of the code will reproduce the code above to obtain a matrix of plots and results illustrating the effect accross columns and cut-off points. 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

cut_offs = [0.1, 0.2, 0.3, 0.5, 0.9]
columns = ['Is Laundering', 'FAN-OUT', 'FAN-IN', 'GATHER-SCATTER', 'SCATTER-GATHER', 'CYCLE', 'RANDOM', 'BIPARTITE', 'STACK']

In [ ]:
def divergence_metric(dist_0, dist_1):
    mean_0 = np.mean(dist_0)
    variance_0 = np.var(dist_0)
    mean_1 = np.mean(dist_1)
    variance_1 = np.var(dist_1)
    return (mean_0 - mean_1)**2 + 0.5*(variance_0 + variance_1)

In [ ]:
n = len(cut_offs)
m = len(columns)

divergence_matrix = np.zeros((n, m))

fig, axes = plt.subplots(n, m, figsize = (n*5, m))

for i in range(n):
    cut_off = cut_offs[i]
    for j in range(m):
        column = columns[j]

        laundering_combined["Label"] = ((laundering_combined[column]>cut_off)*1).values

        labels = []
        for node in results_df.index:
            label = int(laundering_combined.loc[node]["Label"])
            labels.append(label)

        results_df["Label"] = labels
    
        # Filter the DataFrame by label
        label_0 = results_df[results_df["Label"] == 0]["GARGAML"]
        label_1 = results_df[results_df["Label"] == 1]["GARGAML"]

        # Calculate the bin edges
        all_data = np.concatenate([label_0, label_1])
        bins = np.histogram_bin_edges(all_data, bins=20)

        # Plot histogram for label 0
        axes[i, j].hist(label_0, bins=bins, alpha=0.5, label='Label 0', density=True)

        # Plot histogram for label 1
        axes[i, j].hist(label_1, bins=bins, alpha=0.5, label='Label 1', density=True)

        # Add labels and title
        #axes[i, j].set_xlabel('GARGAML')
        #axes[i, j].set_ylabel('Relative Frequency')
        #axes[i, j].set_title('Label: "'+ column +'" at '+ str(cut_off))
        #axes[i, j].legend()

        divergence = divergence_metric(label_0, label_1)
        divergence_matrix[i, j] = divergence

divergence_df = pd.DataFrame(divergence_matrix, columns=columns, index=cut_offs)
divergence_df.to_csv("results/"+dataset+"_GARGAML_"+str_directed+"_combined_divergence.csv")

for axis, col in zip(axes[0], columns):
    axis.set_title(col)

for axis, row in zip(axes[:,0], cut_offs):
    axis.set_ylabel(row, size='large')

fig.suptitle('Distribution of '+ str_directed +' GARGAML Scores by Label for data set: '+ dataset)
fig.tight_layout()

plt.savefig("results/"+dataset+"_GARGAML_"+str_directed+"_combined_histogram.pdf")

In [ ]:
n = len(cut_offs)
m = len(columns)

fig, axes = plt.subplots(n, m, figsize = (n*5, m))
values = np.linspace(0.01, 1, 100)

for i in range(n):
    cut_off = cut_offs[i]
    for j in range(m):
        column = columns[j]

        laundering_combined["Label"] = ((laundering_combined[column]>cut_off)*1).values

        labels = []
        for node in results_df.index:
            label = int(laundering_combined.loc[node]["Label"])
            labels.append(label)

        results_df["Label"] = labels
    
        lift = lift_curve_values(results_df["Label"], results_df["GARGAML"], values)

        axes[i, j].plot(values, lift)

for axis, col in zip(axes[0], columns):
    axis.set_title(col)

for axis, row in zip(axes[:,0], cut_offs):
    axis.set_ylabel(row, size='large')

fig.suptitle('Lift curve of '+ str_directed +' GARGAML Scores by Label for data set: '+ dataset)
fig.tight_layout()

plt.savefig("results/"+dataset+"_GARGAML_"+str_directed+"_combined_lift.pdf")

## Divergence

In [1]:
import os
import sys

DIR = "../"
os.chdir(DIR)
sys.path.append(DIR)

In [2]:
import seaborn as sns
import matplotlib.pylab as plt
import pandas as pd

In [3]:
dataset = "HI-Small"  
directed = False
str_directed = "directed" if directed else "undirected"
score_type = "basic"

In [4]:
cut_offs = [0.1, 0.2, 0.3, 0.5, 0.9]
columns = ['Is Laundering', 'FAN-OUT', 'FAN-IN', 'GATHER-SCATTER', 'SCATTER-GATHER', 'CYCLE', 'RANDOM', 'BIPARTITE', 'STACK']

In [5]:
divergence = pd.read_csv("results/"+dataset+"_GARGAML_"+str_directed+"_combined_divergence.csv", index_col=0)

In [ ]:
with sns.axes_style("white"):
    ax = sns.heatmap(divergence, cmap="YlGnBu", annot=True, fmt=".2f")
    plt.ylabel('Cut-off')
    plt.xlabel('Pattern')
    plt.tight_layout()
    plt.show()

In [8]:
datasets = ["HI-Small", "LI-Large"]
directeds = [False, True]

for dataset in datasets:
    for directed in directeds:
        str_directed = "directed" if directed else "undirected"
        score_type = "basic"

        divergence = pd.read_csv("results/"+dataset+"_GARGAML_"+str_directed+"_combined_divergence.csv", index_col=0)

        with sns.axes_style("white"):
            ax = sns.heatmap(divergence, cmap="YlGnBu", annot=True, fmt=".2f")
            plt.title("Divergence metric of "+ str_directed +" GARGAML Scores for data set: "+ dataset)
            plt.ylabel('Cut-off')
            plt.xlabel('Pattern')
            plt.tight_layout()
            plt.savefig("results/"+dataset+"_"+str_directed+"_divergence_heatmap.pdf")
            plt.close()
        

# Performance Metrics

This part of this notebook deals with the tree-based models. Below, we load the performance metrics as calculated by `gargaml_tree.py` and plot the results in a heat map.

In [1]:
import os
import sys

DIR = "../"
os.chdir(DIR)
sys.path.append(DIR)

In [2]:
import seaborn as sns
import matplotlib.pylab as plt
import pandas as pd

In [8]:
dataset = "LI-Large"  
directed = True
str_directed = "directed" if directed else "undirected"
score_type = "basic"

In [9]:
metrics = pd.read_csv("results/"+dataset+"_imbalance_"+str_directed+"_combined.csv", index_col=0)
metrics_p = metrics*100

In [ ]:
with sns.axes_style("white"):
    ax = sns.heatmap(metrics_p, cmap="YlGnBu", annot=True, fmt=".2f")
    plt.ylabel('Cut-off')
    plt.xlabel('Pattern')
    plt.tight_layout()
    plt.show()

In [ ]:
print(metrics_p.T.to_latex())

In [11]:
models = ["AUC_ROC_tree", "AUC_PR_tree", "AUC_ROC_boosting", "AUC_PR_boosting"]

for model in models: 
    metrics = pd.read_csv("results/"+dataset+"_"+model+"_"+str_directed+"_combined.csv", index_col=0)
    metrics_p = metrics*100

    model_split = model.split("_")

    with sns.axes_style("white"):
        ax = sns.heatmap(metrics_p, cmap="YlGnBu", annot=True, fmt=".2f",  
                         cbar_kws={'label': model_split[0]+"-"+model_split[1]})
        ax.set_xticklabels(ax.get_xticklabels(), rotation=70)
        title_string = "Gradient boosting" if model_split[2] == "boosting" else "Decision tree"
        plt.title(title_string)
        plt.ylabel('Cut-off')
        plt.xlabel('Pattern')
        plt.tight_layout()
        plt.savefig("results/"+dataset+"_"+model+"_"+str_directed+"_heatmap.pdf")
        plt.close()